# Land degradation <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2), 
[s2_12a](https://explorer.digitalearth.africa/s2_12a)

## Background

[Surface mining](https://americanmineservices.com/types-of-surface-mining) refers to the removal of the terrain surface to access minerals underneath. In particular, surface mining is used to retrieve sand, gravel, stones, coal, iron and other metals. Surface mining is often more cost-effective than gouging tunnels and subterranean shafts to access minerals underground.

Although surface mining contributes to the source of income for a country, illegal mining operations can result in deleterious impacts on farmlands, forests, and water bodies. Government officials are making efforts to identify areas of illegal mining activities and to restore the mining sites. This notebook demonstrates a method for identifying regions where land degradation and water loss has occurred. This may indicate mining is taking place. The regions identified do not necessarily identify areas of illegal mining activities, further verification by government agencies is required to discern illegal vs legal mining operations.


## Description

Mining operations often result in the clearing of vegetation and the develpment of water bodies for processing ore. These land-use changes can be identified by finding region where changes in vegetation and water extents occurr. Using remote sensing images Sentinel-2 and DE Africa's Water Observations from Space (WOfS) product, this notebook finds regions where coincident changes in vegetation and water extent occur. The coupling of these two changes may reveal locations of illegal mining, or general land degradation. However, the user should not accept the accuracy of the results but should conduct ground validation testing to assess accuracy. 

In most cases, these algorithms can be used to identify clusters of pixels that have experienced change and allow targeted investigation of those areas by local or regional governments.

The notebook demonstrates how to:

1. Load data for a given location and time period
2. Calculate the vegetation change using NDVI
3. Compare the water extent for two time periods
4. Combine vegetation change and water extent change into one plot to show possible illegal mining areas

***

In [1]:
%matplotlib inline

import sys
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.cm import RdYlGn
from datacube.utils import geometry
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from datacube.utils.cog import write_cog

sys.path.append('../Scripts')
from odc.algo import xr_geomedian
from deafrica_bandindices import calculate_indices
from deafrica_dask import create_local_dask_cluster
from deafrica_datahandling import load_ard, wofs_fuser, mostcommon_crs
from deafrica_plotting import display_map, map_shapefile, rgb
from deafrica_spatialtools import xr_rasterize

### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [2]:
dc = datacube.Datacube(app='Land_degradation')

### Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To activate Dask, set up the local computing cluster using the cell below.

In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:35449 Dashboard: /user/nanaboamah/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
There is also a parameter to define how the data is split in time. Note you must specify either the path of the shapefile or use the lattitude, longitude and buffer.

The parameters are:

* `vector_file`: The file path and name of a shapefile containing the extent. 
* `latitude`: The latitude at the centre of your Area of Interest (AOI) (e.g. `0.02`).
* `longitude`: The longitude at the centre of your AOI (e.g. `35.425`).
* `buffer`: The number of degrees to load around the central latitude and longitude.
For reasonable loading times, set this as `0.1` or lower.
* `time`: The date range to analyse (e.g. `('2015-01-01', '2019-09-01')`).
For reasonable results, the range should span at least two years to minimise the impact of seasonal changes.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example covers part of the Northern Tindiret Forest Reserve, Kenya, and uses the shapefile provided in **Supplementary Data**.

In [4]:
baseline_year = 2013
analysis_year = 2017

# vector_file = None
vector_file = "data/upperdenkyira.shp"

if vector_file:
    # Read shapefile into a GeoDataFrame
    gdf = gpd.read_file(vector_file)

    # Convert all of the shapes into a datacube geometry
    geom = geometry.Geometry(gdf.unary_union, gdf.crs)
    
else:
    longitude = -1.892
    latitude = 6.31744
    buffer = 0.05
    
    lat_range = (latitude-buffer, latitude+buffer)
    lon_range = (longitude-buffer, longitude+buffer)

The code below renders a map that can be used to orient yourself with the region.

In [5]:
if not vector_file:
    display_map(lon_range, lat_range)
else:
    map_shapefile(gdf, attribute=gdf.columns[0], fillOpacity=0, weight=3)

Label(value='')

Map(center=[6.106385415185879, -1.9992221151967147], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Load and clean data from the Data Cube 
    
The first step in the analysis is to load Landsat data for the specified area of interest and time range. 

The code below will create a query dictionary for our region of interest, find the correct `crs` object for the area of interest, and then load the Landsat data using the `load_ard` function.
For more information, see the [Using load_ard notebook](../Frequently_used_code/Using_load_ard.ipynb).

In [6]:
#Create a query object
query = {
    'group_by': 'solar_day',
}

if vector_file:
    query["geopolygon"] = geom
else:
    query["x"] = lon_range
    query["y"] = lat_range

query['resolution'] = (-10, 10)

product = 'ls8_usgs_sr_scene' #'s2_l2a'
# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product=product, query=query)

ds = load_ard(dc=dc,
              products=[product], output_crs=output_crs,
              dask_chunks={'time':1,'x':2000,'y':2000},
              measurements=["red","green","blue","nir","swir1","swir2"],
              time=(f'{baseline_year}', f'{analysis_year}'),
              scaling='normalised',
              **query)

Using pixel quality parameters for USGS Collection 1
Finding datasets
    ls8_usgs_sr_scene
Applying pixel quality/cloud mask
Re-scaling Landsat C1 data
Returning 137 time steps as a dask array


### Compute geomedian for each year
For more information about computing geomedians, see the [Generating geomedian composites](../Frequently_used_code/Generating_geomedian_composites.ipynb) notebook.

In [7]:
ds_geomedian = ds.groupby('time.year').map(xr_geomedian)

ds_geomedian = ds_geomedian.sel(year=[baseline_year, analysis_year])

### Calculate indices for each year

Using the `calculate_indices` functions, NDVI and MNDWI is computed on the geomedian dataset.

In [8]:
ds_geomedian = calculate_indices(ds_geomedian, ['NDVI','MNDWI','WI'], collection='c1')

The next line of code will trigger the delayed computations up to the point using Dask's `.compute()` method. To check the progress of the calculations use the hyperlink printed below the `create_local_dask_cluster()` cell at the top of this notebook.

In [ ]:
ds_geomedian = ds_geomedian.compute()

## Apply a mask to each year
We will use the polygon from the shapefile to create a mask, so we can work with only the pixels in the area. To convert the vector polygon to a raster mask, we use `xr_rasterize`. For more information on indices, see the [Rasterising vectors & vectorising rasters](/Frequently_used_code/Rasterise_vectorise.ipynb) notebook.

In [ ]:
# Convert the polygon to a raster that matches our imagery data
if vector_file:
    mask = xr_rasterize(gdf, ds_geomedian)

    # Mask dataset to set pixels outside the polygon to `NaN`
    ds_geomedian = ds_geomedian.where(mask)

### Calculate the anomaly in vegatation for the selected years

The anomaly is calculated by subtracting the analysis year data from the baseline data to identifying the change over the two periods.
A `vegetation_loss_threshold` is set to pick out areas that vegetation was loss. The threshold can be varied depending on the area of interest.

In [ ]:
baseline = ds_geomedian.isel(year=0)
analysis = ds_geomedian.isel(year=1)

parameter_anomaly = analysis.NDVI - baseline.NDVI


vegetation_loss_threshold = -0.2
# Determine areas with significant deforestation (negative difference)
vegetation_loss = parameter_anomaly < vegetation_loss_threshold
vegetation_loss['name'] = "Vegetation_loss"

# Determine areas with significant afforestations (positive difference)
vegetation_gain = parameter_anomaly > 0.2
vegetation_gain['name'] = "Vegetation_gain"

## Plot baseline RGB, analysis RGB and anomaly products

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(18, 16))
baseline[['red', 'green', 'blue']].to_array().plot.imshow(ax=ax[0,0], robust=True)
analysis[['red', 'green', 'blue']].to_array().plot.imshow(ax=ax[0,1], robust=True)
parameter_anomaly.plot(cmap='RdYlGn', ax=ax[1,0], add_colorbar=False)
vegetation_loss.plot(cmap='OrRd', ax=ax[1,1],  add_colorbar=False)
ax[0,0].set_title('Baseline mean composite')
ax[0,1].set_title('Analysis composite')
ax[1,0].set_title('Vegetation Anomalies: Red=Loss, Green=Gain')
ax[1,1].set_title('Locations of Significant Anomalies-Vegetation loss: Red=Loss')

plt.show()

### Calculate the area per pixel

In [ ]:
pixel_length = query["resolution"][1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres
area_per_pixel = pixel_length**2 / m_per_km**2

### Calculate total area of pixel, area and percentage of vegetation loss pixels

In [ ]:
total_pixels = analysis.NDVI.count(dim=['x', 'y']).values
total_vegetation_loss = vegetation_loss.where(vegetation_loss==True).count(dim=['x', 'y']).values

total_forest_area = total_pixels * area_per_pixel
vegetation_loss_area = total_vegetation_loss * area_per_pixel
percentage_vegetation_loss = (total_vegetation_loss/total_pixels) * 100

print(f"Total Area: {total_forest_area:.2f} km sq")
print(f"Vegetation Loss : {vegetation_loss_area:.2f} km sq")
print(f"{percentage_vegetation_loss:.2f}% of pixels likely underwent vegetation loss")

## Determine the change in water extent using WOFS and MNDWI

### Load WOfS data

As we want the time and spatial extents to match the vegetation change data, we can use the same query defined above.

In [ ]:
query['resolution'] =  (-30, 30)

# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product=["ga_ls8c_wofs_2"], query=query)

query['time'] = (f'{baseline_year}')
ds_wofs_baseline = dc.load(product=["ga_ls8c_wofs_2"],
             output_crs=output_crs,
             fuse_func=wofs_fuser,
             **query
            )

query['time'] = (f'{analysis_year}')
ds_wofs_analysis = dc.load(product=["ga_ls8c_wofs_2"],
             output_crs=output_crs,
             fuse_func=wofs_fuser,
             **query
            )

### Select "water" pixels from the WOFS dataset

WOfS uses bit flags to identify water and non-water pixels. Water pixels have the value `128`. See [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) for more information.

In [ ]:
ds_water_baseline = ds_wofs_baseline.water.isin([128])

ds_water_analysis = ds_wofs_analysis.water.isin([128])

if vector_file:
    ds_water_baseline = ds_water_baseline.where(mask)
    
    ds_water_analysis = ds_water_analysis.where(mask)

### Calculate the area per pixel

In [ ]:
pixel_length = query["resolution"][1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres
area_per_pixel = pixel_length**2 / m_per_km**2

### Calculate area covered by water

In [ ]:
ds_water_area_baseline = ds_water_baseline.sum(dim=['x', 'y']) * area_per_pixel

ds_water_area_analysis = ds_water_analysis.sum(dim=['x', 'y']) * area_per_pixel

### Select maximum water observed for the two years

In [ ]:
baseline_max_water, analysis_max_water = max(ds_water_area_baseline),  max(ds_water_area_analysis)

max_water_baseline = ds_water_baseline.sel(time=baseline_max_water.time.values)

max_water_analysis = ds_water_analysis.sel(time=analysis_max_water.time.values)

In [ ]:
# The dataset array is transform to 1 and 0 using the `astype(int)` function.
change = max_water_analysis.astype(int) - max_water_baseline.astype(int)

# A threshold greater than 0 is  for area where MNDWI indices is water
threshold_value = 0

# We set areas where threshold greater than 0 to 1 and others to 0
analysis_mndwi = analysis.MNDWI >= threshold_value
baseline_mndwi = baseline.MNDWI >= threshold_value

#The change is calculated to determine water gain, loss and permanent
change_mndwi = analysis_mndwi.astype(int) - baseline_mndwi.astype(int)

In [ ]:
# A threshold greater than 0 is  for area where MNDWI indices is water
threshold_value = 1.720

# We set areas where threshold greater than 0 to 1 and others to 0
analysis_wi = analysis.WI >= threshold_value
baseline_wi = baseline.WI >= threshold_value
change_wi = analysis_wi.astype(int) - baseline_wi.astype(int)

### Plotting of Results
When water(WOFS, WI, MNDWI) and vegetation loss superimposed, the area is classified as Highly potential mining site.
For Areas where there was only vegetation loss there is a meduim potential of identifying mining sites at that area.

In [ ]:
plt.figure(figsize=(12, 12))
vegetation_loss = vegetation_loss.where(vegetation_loss==True)
mining_area = vegetation_loss.where(change_wi | change | change_mndwi)

analysis.NDVI.plot.imshow(cmap='Greys', add_colorbar=False)

vegetation_loss.plot.imshow(cmap=ListedColormap(['Brown']), add_colorbar=False)
mining_area.plot.imshow(cmap=ListedColormap(['Gold']), add_colorbar=False)


plt.legend(
        [Patch(facecolor='Gold'), Patch(facecolor='Brown')], 
        ['Highly Potential Mining Site', 'Medium Potential Mining Site'],
        loc = 'upper right'
    )

plt.title('Possible Mining Areas')

### Downlaod the end product

In [ ]:
write_cog(deforestaion.astype(float), fname='deforestaion.tif', overwrite=True)
write_cog(mining_area.astype(float), fname='mining_area.tif', overwrite=True, nodata=0)


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** November 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)